In [17]:
from keras.models import Sequential
from keras.layers import Dropout, Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
import matplotlib.pyplot as plt

In [18]:
def create_model(num_classes):
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(100,100,1), activation="relu"))
    model.add(Conv2D(32, (5, 5), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(64, (3, 3), activation="relu"))
    model.add(Conv2D(64, (3, 3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation="softmax"))
    model.compile(optimizer=RMSprop(learning_rate=0.001, rho=0.9, epsilon=1e-08, decay=0.0), loss="categorical_crossentropy", metrics=["accuracy"])
    return model

In [19]:
import im_processor

In [20]:
data_gen = ImageDataGenerator(
    preprocessing_function=im_processor.preprocess_img,
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range = 0.1, # Randomly zoom image
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images
    vertical_flip=False,
    validation_split=0.1)

train_data = data_gen.flow_from_directory('dataset/', target_size=(100,100), color_mode='grayscale', class_mode='categorical', batch_size=32, subset='training', shuffle=True)

val_data = data_gen.flow_from_directory('dataset/', target_size=(100,100), color_mode='grayscale', class_mode='categorical', batch_size=32, subset='validation', shuffle=True)

Found 9072 images belonging to 19 classes.
Found 999 images belonging to 19 classes.


In [21]:
train_data.image_shape

(100, 100, 1)

In [22]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=3,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00001)

In [23]:
learning_model = create_model(num_classes=19)

In [24]:
epochs = 32
history = learning_model.fit(train_data,steps_per_epoch=train_data.samples,validation_data=val_data,validation_steps=val_data.samples,epochs=epochs)

Epoch 1/32
 264/9072 [..............................] - ETA: 11:51 - loss: nan - accuracy: 0.0593

2022-04-11 23:09:08.847373: W tensorflow/core/framework/op_kernel.cc:1733] UNKNOWN: IndexError: list index out of range
Traceback (most recent call last):

  File "/home/mateo/.local/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 271, in __call__
    ret = func(*args)

  File "/home/mateo/.local/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/home/mateo/.local/lib/python3.10/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 1004, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/home/mateo/.local/lib/python3.10/site-packages/keras/engine/data_adapter.py", line 830, in wrapped_generator
    for data in generator_fn():

  File "/home/mateo/.local/lib/python3.10/site-packages/keras/engine/data_adapter.py", line 956, in generator_fn
    yield x[i]

  File "/home/mateo/.local/lib/python3.10/site-packages/keras_preprocessing/im

UnknownError: Graph execution error:

2 root error(s) found.
  (0) UNKNOWN:  IndexError: list index out of range
Traceback (most recent call last):

  File "/home/mateo/.local/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 271, in __call__
    ret = func(*args)

  File "/home/mateo/.local/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/home/mateo/.local/lib/python3.10/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 1004, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/home/mateo/.local/lib/python3.10/site-packages/keras/engine/data_adapter.py", line 830, in wrapped_generator
    for data in generator_fn():

  File "/home/mateo/.local/lib/python3.10/site-packages/keras/engine/data_adapter.py", line 956, in generator_fn
    yield x[i]

  File "/home/mateo/.local/lib/python3.10/site-packages/keras_preprocessing/image/ite

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()